# `__slots__`
changes the way that objects are stored in memory

If you are new to python you don't need to care.

In [ ]:
import sys


def get_size(object):
    """Gets a rough estimate of the size of an object in memory."""
    size = sys.getsizeof(object)
    try:
        size += sys.getsizeof(object.__dict__)
        size += sys.getsizeof(object.__weakref__)
    except AttributeError:
        pass
    return size

firs thing

In [ ]:
class NoSlots:
    """User defined class with out __slots__"""

    
no_slots = NoSlots()
get_size(no_slots)

In [ ]:
class WithSlots:
    """User defined class with __slots__"""
    __slots__ = ()

with_slots = WithSlots()
get_size(with_slots)

Objects that have `__slots__` take up less memory. 

Why is this?

What does `no_slots` have that `with_slots` doesn't have?

In [ ]:
no_slots_attrs = set(dir(no_slots))
with_slots_attrs = set(dir(with_slots))

print(no_slots_attrs - with_slots_attrs)

Normal objects have a `dict` that alows them to acomodate an arbitrary number of attributes.

What does 'with_slots' have that `no_slots` doesn't have?

In [ ]:
print(with_slots_attrs - no_slots_attrs)

Objects with `__slots__` reserve exaclty the amout of memory that they will need for attributes. 

Assigning new attributes to a typical object works fine.

In [ ]:
no_slots.arbitrary_attribute = 5

Trying to assign an unexpected attribute to an object that has `__slots__` doesn't work.

In [ ]:
with_slots.arbitrary_attribute = 5

The `WithSlots` class is rather useless because it doesn't allow any attributes at all.

In [ ]:
class Point:
    """A point in 2D space"""
    __slots__ = ('x', 'y')

Space for the `x` and `y` attributes is reserved in memory so we can assign to them.

In [ ]:
origin = Point()
origin.x = 0
origin.y = 0

You can't make the point 3D by adding a new `z` attribute 
because there is no memory reserved for `z`.

In [ ]:
origin.z = 0

How does the number of attribures affect memory consumption?

In [ ]:
class LotsOfAttributes:
    __slots__ = ('a', 'b', 'c', 'd', 'e', 'f')
    
lots_of_attributes = LotsOfAttributes()
get_size(lots_of_attributes)

In [ ]:
no_slots = NoSlots()
no_slots.a = 1
no_slots.b = 2
no_slots.c = 3
no_slots.d = 4
no_slots.e = 5
no_slots.f = 5

get_size(no_slots)

`__slots__` is ideal for objects that you will create a lot of.

How does `__slots__` work with inheritance?

In [ ]:
class AddSlots(NoSlots):
    """Define __slots__ and inherit from a class that doesn't have __slots__."""
    __slots__ = ()

add_slots = AddSlots()
get_size(add_slots)

In [ ]:
class RemoveSlots(WithSlots):
    """Don't define __slots__ and inherit from a class that has __slots__."""

remove_slots = RemoveSlots()
get_size(remove_slots)

Every ancestor must define `__slots__` to get the memory benifites.

Multiple inheritance

In [ ]:
class Pointless(Point, WithSlots):
    __slots__ = ()

In [ ]:
class MashUp(Point, LotsOfAttributes):
    __slots__ = ()

Sloted attributes are treated as data descriptors which makes looking them up a little faster.

# Design Considerations

Start a library without using `__slots__`.

In [ ]:
# library code
class Point:
    """A point in 2D space."""
    
    def __init__(self, x, y):
        self.x, self.y = x, y

In [ ]:
# project that uses library
p = Point(x=1, y=2)
p.z = 3

Add `__slots__` to the library.

In [ ]:
# library code
class Point:
    """A point in 2D space."""
    __slots__ = ('x', 'y')
    
    def __init__(self, x, y):
        self.x, self.y = x, y

In [ ]:
# project that uses library
p = Point(x=1, y=2)
p.z = 3

Adding slots to a class in a library is a breaking change!

Start a library with `__slots__`.

In [ ]:
# library code
class Point:
    """A point in 2D space."""
    __slots__ = ('x', 'y')
    
    def __init__(self, x, y):
        self.x, self.y = x, y

In [ ]:
# project that uses the library
p = Point(1, 2)

Remove __slots__ from library

In [ ]:
# library code
class Point:
    """A point in 2D space."""
    
    def __init__(self, x, y):
        self.x, self.y = x, y

In [ ]:
# project code doen't break
p = Point(1, 2)

# Warning!
`__slots__` is an optimization.

Premature optimization is usually a bad idea.

## References
* [Python Data Model](https://docs.python.org/3/reference/datamodel.html)
* [Slot or Not](https://www.youtube.com/watch?v=N7MfisN44nY&t=18s)